In [1]:
import os
import sys

# Set environment variables before importing transformers
os.environ["HUGGINGFACE_HUB_CACHE"] = "/scratch/shareddata/dldata/huggingface-hub-cache/hub" # Load local models
os.environ["TRANSFORMERS_OFFLINE"] = "1" 
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["HF_HOME"] = f"{os.environ["WRKDIR"]}/.cache/huggingface" # Cache in work directory

sys.path.append(
    os.path.abspath(os.path.join(os.getcwd(), ".."))
)  # Add parent directory to path

import json
import pandas as pd
import transformers
from datasets import Dataset

from utils.prompts import (
    JUDGE_SYSTEM_PROMPT,
    JUDGE_TEMPLATE,
)

In [2]:
DEFAULT_DATA = r"../../data/complete_dataset.csv"
DEFAULT_MODEL = "Qwen/Qwen2.5-14B-Instruct"

system_prompts = {
    "judge": JUDGE_SYSTEM_PROMPT,
}

In [3]:
def make_prompt(row, task_type):
    _, problem_description, example_solution, _, _, theme, topic, concept, *_ = row

    match task_type:
        case "judge":
                return (JUDGE_TEMPLATE.replace("$THEME$", theme)
                .replace("$TOPIC$", topic)
                .replace("$CONCEPT$", concept)
                .replace("$TEXT$", problem_description)
                .replace("$CODE$", example_solution)
            )
        case _:
            raise ValueError(f"Task type '{_}' not recognised as valid task type!")


def run_model(pipe, data, task_type):
    system_prompt = system_prompts.get(task_type, None)

    if system_prompt is None:
        raise ValueError(f"Task type '{task_type}' not recognised as valid task type!")

    response = pipe(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": data["prompt"]},
        ],
        return_full_text=False,
        max_new_tokens=500,
    )

    result = response[0]["generated_text"]  
    result_dict = json.loads(result)

    for k, v in result_dict.items():
        data[k] = v

    return data

In [4]:
task = "judge"

In [5]:
params = {
    "model": DEFAULT_MODEL,
    "device_map": 0,  # Force GPU
    "max_new_tokens": 500,
    "temperature": 0.3,
}

pipeline = transformers.pipeline("text-generation", **params)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0


In [6]:
df = pd.read_csv(DEFAULT_DATA, sep=";")
n_rows = 3
eval_df = df
#eval_df = df.loc[0:n_rows - 1]
eval_df["prompt"] = eval_df.apply(lambda row: make_prompt(row, task), axis=1)

In [7]:
columns = df.columns
#print(columns)

eval_df = eval_df[eval_df[columns[11]] == "no"] # The exercise description does not match selected theme

eval_df

,title,problemDescription,exampleSolution,starterCode,tests,theme,topic,concept,difficulty,id,The exercise description was clear (Yes/Partially/No),The exercise description matched the selected theme (Yes/Partially/No),The exercise description matched the selected topic (Yes/Partially/No),The exercise description matched the selected concept (Yes/No),Included concepts that were too advanced (Yes/No)\n,The exercise difficulty matched the selected difficulty (Too easy/Okay/Too difficult),Shallow vs deep personalization (Deep/Shallow/Unsure),Open field,prompt
83,Berry Picking!,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...","{'code': ""import 'dart:io';main() {}""}","{'testCode': ""import 'package:test/test.dart';...",outdoor activities,berry picking,user input,normal,3.688261e+14,yes,no,no,yes,no,okay,shallow,NaN,You are evaluating a programming exercise.\n\n...
121,Pass the Parcel!,Write a program that asks the user for their n...,"{'code': ""import 'dart:io';main() { print('Wh...","{'code': ""import 'dart:io';main() {}""}","{'testCode': ""import 'package:test/test.dart';...",party games,Pass the Parcel,user input,advanced,9.796044e+14,yes,no,no,yes,no,too easy,shallow,NaN,You are evaluating a programming exercise.\n\n...
158,Pass the Parcel!,Write a program that asks the user for their n...,"{'code': ""import 'dart:io';main() { print('Wh...","{'code': ""import 'dart:io';main() {}""}","{'testCode': ""import 'package:test/test.dart';...",party games,Pass the Parcel,program output,normal,6.068320e+14,yes,no,yes,yes,no,too difficult,unsure,NaN,You are evaluating a programming exercise.\n\n...
220,Holiday Parade Attendance,Write a program that asks the user if they wil...,"{'code': ""import 'dart:io'; main() { print(...","{'code': ""import 'dart:io'; main() { }""}","{'testCode': ""import 'package:test/test.dart';...",Christmas,attending holiday parades,program output,normal,-8.374976e+13,yes,no,yes,yes,yes,too difficult,unsure,NaN,You are evaluating a programming exercise.\n\n...


In [8]:
result = eval_df.apply(lambda row: run_model(pipeline, row, task), axis=1)
print("Done")

Done


In [9]:
for idx, row in result.iterrows():
    print("Theme: ", row["ThemeCorrect"])
    print("Topic: ", row["TopicCorrect"])
    print("Concept: ", row["ConceptCorrect"])
    print("Reasoning: ", row["Explanation"])
    print("")

Theme:  partially
Topic:  yes
Concept:  yes
Reasoning:  The exercise is about outdoor activities, specifically focusing on berry picking, which aligns well with the intended theme and topic. However, the theme 'outdoor activities' is quite broad and could include many other activities not related to berries, so it's only partially correct. The topic 'berry picking' is directly addressed in the exercise. The exercise correctly uses user input to gather information from the user and then outputs a personalized message based on that input, matching the intended programming concept.

Theme:  no
Topic:  no
Concept:  yes
Reasoning:  The exercise does not align with the intended theme of 'party games' or the specific topic of 'Pass the Parcel'. However, it correctly focuses on the programming concept of user input, as it requires the user to enter their name and favorite color.

Theme:  partially
Topic:  partially
Concept:  yes
Reasoning:  The exercise is about getting user input for their na